In [ ]:
!pip install opendatasets

In [ ]:
import sqlalchemy
import pandas as pd
import opendatasets as od

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///sql_test.db')
sqlite_connection = engine.connect()
sqlite_table = 'movies'

In [ ]:
def run_query(query, engine):
    return pd.read_sql(query, con=engine, index_col=None)

In [ ]:
od.download('https://www.kaggle.com/datasets/anotherbadcode/boxofficecollections')

In [ ]:
df = pd.read_csv('./boxofficecollections/BoxOfficeCollections.csv')

In [ ]:
df.columns = [x.lower().replace(' ', '') for x in df.columns.tolist()]

In [ ]:
df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

# SQL Test

#### What is the name of the movie that has the highest IMDB rating in 2017 ?

In [ ]:
query = '''
select
    movie,
    year,
    max(imdbrating) as max_rating
from movies
where year = 2017
'''
run_query(query, engine)

#### How many comedy/action/sci-fi ... films are made each year ?

In [ ]:
query = '''
select 
    year,
    imdb_genre,
    count(imdb_genre) as count_per_year
from movies
group by year, imdb_genre
order by year

'''
run_query(query, engine)

#### Select those comedy movies made in 2020 that are rated higher than the average comedy movie (based on imdb rating)

In [ ]:
query = '''
select
    *
from movies
where
    (imdb_genre = "Comedy" and year = 2020)
and
    (imdbrating > (select avg(imdbrating) from movies))

'''
run_query(query, engine)

#### List the best movies from every year in a chronological order (based on imdb rating)

In [ ]:
query = '''
select
    movie,
    year,
    max(imdbrating) as max_rating
from movies
group by year
order by year

'''
run_query(query, engine)

#### Which director has worked with the highest number of actors ?

In [ ]:
query = '''
select
    director,
    sum((select
            (length("cast") - length(replace("cast", "," , "" )) + 1) as num_of_actors
         from movies)) as actors_sum
from movies
group by director
order by actors_sum desc
limit 1

'''
run_query(query, engine)